In [1]:
# imports

import os
import io
import sys
from openai import OpenAI
import gradio as gr
import subprocess
from IPython.display import Markdown, display

In [2]:
# Connect to client libraries

openai = OpenAI()
ollama_url = "http://localhost:11434/v1"
ollama = OpenAI(api_key="ollama", base_url=ollama_url)

In [3]:
models = ["gpt-oss:20b", "llama3.2:latest", "llama3.1:8b", "gemma3:270m",]
clients ={"gpt-oss:20b":ollama, "llama3.2:latest":ollama, "llama3.1:8b":ollama, "gemma3:270m":ollama}

In [5]:
from system_info import retrieve_system_info

system_info = retrieve_system_info()
system_info

{'os': {'system': 'Windows',
  'arch': 'AMD64',
  'release': '11',
  'version': '10.0.26200',
  'kernel': '11',
  'distro': None,
  'wsl': False,
  'rosetta2_translated': False,
  'target_triple': 'mingw32'},
 'package_managers': ['winget'],
 'cpu': {'brand': 'AMD Ryzen 7 5800HS with Radeon Graphics',
  'cores_logical': 16,
  'cores_physical': 8,
  'simd': []},
 'toolchain': {'compilers': {'gcc': 'gcc.exe (MinGW.org GCC-6.3.0-1) 6.3.0',
   'g++': 'g++.exe (MinGW.org GCC-6.3.0-1) 6.3.0',
   'clang': '',
   'msvc_cl': ''},
  'build_tools': {'cmake': '', 'ninja': '', 'make': ''},
  'linkers': {'ld_lld': ''}}}

In [6]:
message = f"""
Here is a report of the system information for my computer.
I want to run a C++ compiler to compile a single C++ file called main.cpp and then execute it in the simplest way possible.
Please reply with whether I need to install any C++ compiler to do this. If so, please provide the simplest step by step instructions to do so.

If I'm already set up to compile C++ code, then I'd like to run something like this in Python to compile and execute the code:
```python
compile_command = # something here - to achieve the fastest possible runtime performance
compile_result = subprocess.run(compile_command, check=True, text=True, capture_output=True)
run_command = # something here
run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
return run_result.stdout
```
Please tell me exactly what I should use for the compile_command and run_command.

System information:
{system_info}
"""

In [7]:
response = ollama.chat.completions.create(model=models[0], messages=[{"role": "user", "content": message}])
display(Markdown(response.choices[0].message.content))

### 1. Is a compiler already on your system?

Your system information shows that **MinGW‑GCC 6.3.0** is already installed:

```text
'g++': 'g++.exe (MinGW.org GCC-6.3.0-1) 6.3.0'
```

You can confirm this from a PowerShell / CMD window:

```powershell
> g++ --version
```

If you get a version number (as above) you’re good to go – *you do **not** need to install another compiler*.

> **But** GCC 6.3 is a bit old. If you care about the very best runtime performance (especially on a Ryzen 5800HS), you might want to upgrade to a newer toolchain (e.g. GCC 10+, Clang 14+, or MSVC).  
>  Installing a newer compiler is optional – it’s only required if you want that extra speed.

---

### 2. Upgrade the compiler (optional)

If you decide to upgrade via winget, the quick one‑liner is:

```powershell
winget install -e --id "MinGW.Compilers"
```

(This installs MinGW‑w64 GCC 11+ and places `g++` in your PATH.)

---

### 3. Python harness to compile **and** run a single file

Below is a *minimal*, *fastest‑in‑practice* sequence for compiling `main.cpp`
with GCC and executing the resulting binary.

```python
import subprocess
import pathlib

# ------------------------------------------------------------------
# Step 1: build the C++ file into a native Win32 .exe
# ------------------------------------------------------------------
compile_command = [
    "g++",
    "main.cpp",                 # input source file
    "-O3",                      # aggressive optimisation
    "-std=c++17",               # language standard (pick the one you need)
    "-march=native",            # tune for your CPU
    "-flto",                    # link‑time optimisation (if supported)
    "-s",                       # strip debug symbols in the final binary
    "-o", "main.exe"            # output binary
]

# Run the compiler
compile_process = subprocess.run(
    compile_command,          # list of command‑line parts
    check=True,               # raise on non‑zero exit
    capture_output=True,      # capture stdout & stderr
    text=True                 # return string, not bytes
)

# compile_process.stdout   <-- usually empty for a successful build
# compile_process.stderr   <-- compiler diagnostics if any

# ------------------------------------------------------------------
# Step 2: execute the freshly compiled binary
# ------------------------------------------------------------------
run_command = [
    pathlib.Path.cwd() / "main.exe"   # absolute path – safer on Windows
]

run_process = subprocess.run(
    run_command,
    check=True,            # raise if the program exits non‑zero
    capture_output=True,
    text=True
)

# ------------------------------------------------------------------
# Result
# ------------------------------------------------------------------
print("Program output:\n", run_process.stdout)
```

#### Why this particular `g++` command gives the best speed?

| Flag | Purpose |
|------|---------|
| `-O3` | Full optimisation set – fastest (but potentially larger) |
| `-march=native` | Generates code tuned to your exact CPU (Ryzen 5800HS – AVX‑512‑like instructions) |
| `-flto` | Performs inter‑module optimisation across the whole binary (GCC 6+ supports it) |
| `-s` | Strips symbols, shrinking the binary, which can help with startup time |
| `-o main.exe` | Guarantees the binary name; eliminates the default `a.exe` confusion |


---

### 4. Quick note on alternative toolchains

If you prefer **MSVC** (likely the fastest on Windows for x86‑64) or **Clang/LLVM**, the same idea applies:

| Tool | Compile command (simplified) | Run command |
|------|----------------------------|------------|
| MSVC (`cl.exe`) | `["cl", "/EHsc", "/O2", "/std:c++17", "/Fe:main.exe", "main.cpp"]` | `["main.exe"]` |
| Clang (`clang++`) | `["clang++", "-O3", "-std=c++17", "-march=native", "-o", "main.exe", "main.cpp"]` | `["main.exe"]` |

Just replace `g++` with the corresponding executable and rerun the Python harness.

---

### 5. Summary

| ✅ | Result |
|---|--------|
| **Compiler installed?** | Yes – MinGW‑GCC 6.3 is already present. |
| **Upgrade needed?** | Not mandatory, but newer GCC/Clang/MSVC will give you the *fastest* possible binaries. |
| **Python compile‑run code** | Provided above – adjust only the compiler if you upgrade. |
| **Execution** | `subprocess.run(["...","main.exe"], …)` – the binary launches immediately. |

Enjoy compiling with the speed‑oriented command set above!

In [8]:
compile_command = [
                "g++",
                "main.cpp",
                "-o",
                "main.exe"
]

run_command = ["./main"]


In [9]:
language = "C++" 
extension = "cpp"

system_prompt = f"""
Your task is to convert Python code into high performance {language} code.
Respond only with {language} code. Do not provide any explanation other than occasional comments.
The {language} response needs to produce an identical output in the fastest possible time.
"""

def user_prompt_for(python):
    return f"""
Port this Python code to {language} with the fastest possible implementation that produces identical output in the least time.
The system information is:
{system_info}
Your response will be written to a file called main.{language} and then compiled and executed; the compilation command is:
{compile_command}
Respond only with {language} code.
Python code to port:

```python
{python}
```
"""

In [10]:
def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)}
    ]
 

In [11]:
def write_output(code):
    with open(f"main.{extension}", "w") as f:
        f.write(code)


In [12]:
def port(model, python):
    client = clients[model]
    response = client.chat.completions.create(model=model, messages=messages_for(python))
    reply = response.choices[0].message.content
    reply = reply.replace('```cpp','').replace('```','')
    return reply

In [13]:
def run_python(code):
    globals_dict = {"__builtins__": __builtins__}

    buffer = io.StringIO()
    old_stdout = sys.stdout
    sys.stdout = buffer

    try:
        exec(code, globals_dict)
        output = buffer.getvalue()
    except Exception as e:
        output = f"Error: {e}"
    finally:
        sys.stdout = old_stdout

    return output

In [14]:
def compile_and_run(code):
    write_output(code)
    try:
        subprocess.run(compile_command, check=True, text=True, capture_output=True)
        run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
        return run_result.stdout
    except subprocess.CalledProcessError as e:
        return f"An error occurred:\n{e.stderr}"

In [15]:
python_hard = """# Be careful to support large numbers

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [16]:
from styles import CSS

with gr.Blocks(css=CSS, theme=gr.themes.Monochrome(), title=f"Port from Python to {language}") as ui:
    with gr.Row(equal_height=True):
        with gr.Column(scale=6):
            python = gr.Code(
                label="Python (original)",
                value=python_hard,
                language="python",
                lines=26
            )
        with gr.Column(scale=6):
            cpp = gr.Code(
                label=f"{language} (generated)",
                value="",
                language="cpp",
                lines=26
            )

    with gr.Row(elem_classes=["controls"]):
        python_run = gr.Button("Run Python", elem_classes=["run-btn", "py"])
        model = gr.Dropdown(models, value=models[0], show_label=False)
        convert = gr.Button(f"Port to {language}", elem_classes=["convert-btn"])
        cpp_run = gr.Button(f"Run {language}", elem_classes=["run-btn", "cpp"])

    with gr.Row(equal_height=True):
        with gr.Column(scale=6):
            python_out = gr.TextArea(label="Python result", lines=8, elem_classes=["py-out"])
        with gr.Column(scale=6):
            cpp_out = gr.TextArea(label=f"{language} result", lines=8, elem_classes=["cpp-out"])

    convert.click(fn=port, inputs=[model, python], outputs=[cpp])
    python_run.click(fn=run_python, inputs=[python], outputs=[python_out])
    cpp_run.click(fn=compile_and_run, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

C:\Users\Rujjul Saha\AppData\Local\Temp\ipykernel_3400\79197944.py:3: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme, css. Please pass these parameters to launch() instead.
  with gr.Blocks(css=CSS, theme=gr.themes.Monochrome(), title=f"Port from Python to {language}") as ui:


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
